In [1]:
import pandas as pd

In [ ]:
s3_audio = "s3://data-for-case-3/audio/Свод по звонкам и инцидентам (валидационный набор).xlsx"

df_calls_val = pd.read_excel(
    s3_audio,
    storage_options={
        "key": "YCB2BAvtu7_kXfgsmzA_0xv4i",
        "secret": "YCNnsH6n4uHb6_OQ5fpkhN8F4ZhgNbZLwJqsccoV",
        "client_kwargs": {
            "endpoint_url": "https://storage.yandexcloud.kz"
        }
    }
)

df_calls_val.head()

In [13]:
import pandas as pd
import joblib
audio_model = joblib.load('catboost_model.pkl')

s3_audio = "s3://data-for-case-3/audio/Свод по звонкам и инцидентам (валидационный набор).xlsx"

df_calls= pd.read_excel(
    s3_audio,
    storage_options={
        "key": "YCB2BAvtu7_kXfgsmzA_0xv4i",
        "secret": "YCNnsH6n4uHb6_OQ5fpkhN8F4ZhgNbZLwJqsccoV",
        "client_kwargs": {
            "endpoint_url": "https://storage.yandexcloud.kz"
        }
    }
)
df_calls.columns = df_calls.iloc[0]      # set first row as header
df_calls = df_calls[1:].reset_index(drop=True)  # drop first row and reset index

metrics_csv = "validation_audio_voice_metrics.csv"

df_metrics = pd.read_csv(metrics_csv)

df_metrics = df_metrics.rename(columns={"filename": "Файлы"})


df = df_calls.merge(df_metrics, on="Файлы", how="left", suffixes=("_x", "_y"))


columns_out = [
    "APPLICATIONID", "ИИН", "Файлы", "Дата и время звонка",
    "Длительность фонограммы", "Направление вызова", "Группа", "credit_rule"
]



mfcc_and_other = []
for i in range(1, 14):
    mfcc_and_other += [f"mfcc_{i}_mean", f"mfcc_{i}_std"]
mfcc_and_other += [
    "chroma_stft", "spectral_centroid", "spectral_bandwidth",
    "spectral_rolloff", "tempo", "zero_crossing_rate", "rms", "duration", "Файлы"
]
columns_out += mfcc_and_other

if "APPLICATIONID_y" in df.columns:
    df = df.drop(columns=["APPLICATIONID_y"])


df_out = df[columns_out]

important_features = [
    'mfcc_1_mean', 'mfcc_1_std', 'mfcc_2_mean', 'mfcc_2_std',
    'mfcc_3_mean', 'mfcc_3_std', 'mfcc_4_mean', 'mfcc_4_std',
    'mfcc_5_mean', 'mfcc_5_std', 'mfcc_6_mean', 'mfcc_6_std',
    'mfcc_7_mean', 'mfcc_7_std', 'mfcc_8_mean', 'mfcc_8_std',
    'mfcc_9_mean', 'mfcc_9_std', 'mfcc_10_mean', 'mfcc_10_std',
    'mfcc_11_mean', 'mfcc_11_std', 'mfcc_12_mean', 'mfcc_12_std',
    'mfcc_13_mean', 'mfcc_13_std', 'chroma_stft', 'spectral_centroid',
    'spectral_bandwidth', 'spectral_rolloff', 'tempo',
    'zero_crossing_rate', 'rms', 'duration'
]


df_model = df_out.copy()
X_val_proba = df_model[important_features]

y_val_proba = audio_model.predict_proba(X_val_proba)[:, 1]
y_val_pred = (y_val_proba >= 0.5).astype(int)
df_result = df_model.loc[X_val_proba.index, ['APPLICATIONID']].copy()
df_result.rename(columns={'APPLICATIONID':'id'},inplace=True)
df_result['isFrod'] = y_val_pred
df_result['probability'] = y_val_proba
assert len(df_model) == len(y_val_proba)
print('Prediction ready!')
print('Fraud/ no fraud cases distribution: ', df_result['isFrod'].value_counts())

df_result.to_csv('result_audio.csv',index=False)

print("✅ Предсказания сохранены в result_audio.csv")


Prediction ready!
Fraud/ no fraud cases distribution:  isFrod
1    2525
0      25
Name: count, dtype: int64
✅ Предсказания сохранены в result_audio.csv
✅ Сформирован файл merged_audio_voice_features.csv


In [15]:
df_result.isFrod.value_counts()

isFrod
1    2525
0      25
Name: count, dtype: int64

In [5]:
print(df.columns.tolist())

['ИИН', 'APPLICATIONID', 'Дата завки', 'Абонент 2 прив2', 'Дата и время звонка', 'Дата звонка', 'Длительность фонограммы', 'Абонент 1', 'Направление вызова', 'Абонент 2', 'Состояние аудиозаписи', 'Фонограмма прослушана', 'Важная фонограмма', 'Помечена на удаление', 'Группа', 'Ключевые слова', 'Имя станции записи', 'Комментарий', 'Подразделение', 'Имя канала записи', 'Файлы', 'Дата время \nинцидента\n(alert_time)', 'status', 'alert_type', 'tags', 'credit_rule', 'mfcc_1_mean', 'mfcc_1_std', 'mfcc_2_mean', 'mfcc_2_std', 'mfcc_3_mean', 'mfcc_3_std', 'mfcc_4_mean', 'mfcc_4_std', 'mfcc_5_mean', 'mfcc_5_std', 'mfcc_6_mean', 'mfcc_6_std', 'mfcc_7_mean', 'mfcc_7_std', 'mfcc_8_mean', 'mfcc_8_std', 'mfcc_9_mean', 'mfcc_9_std', 'mfcc_10_mean', 'mfcc_10_std', 'mfcc_11_mean', 'mfcc_11_std', 'mfcc_12_mean', 'mfcc_12_std', 'mfcc_13_mean', 'mfcc_13_std', 'chroma_stft', 'spectral_centroid', 'spectral_bandwidth', 'spectral_rolloff', 'tempo', 'zero_crossing_rate', 'rms', 'duration']


In [7]:
import joblib


✅ Предсказания сохранены в final_prediction_output.csv


/tmp/ipykernel_22660/466889222.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_out["fraud_probability"] = y_val_proba
/tmp/ipykernel_22660/466889222.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_out["fraud_prediction"] = y_val_pred
